# 데이터셋 json파일 -> csv or tsv로 추출하기

In [4]:
import os
import json
import pandas as pd
from datetime import datetime

## 선택한 폴더 데이터(json)를 tsv로 변환

In [2]:
# 폴더 리스트 설정
folder_list = [
    "01.news_r",
    "02.briefing",
    "03.his_cul",
    "04.paper",
    "05.minute",
    "06.edit",
    "07.public",
    "08.speech",
    "09.literature",
    "10.narration"
]

# 기본 경로 설정
train_folder_path = 'C:/Users/kmn27/Downloads/01.데이터/1.Training/라벨링데이터'
valid_folder_path = 'C:/Users/kmn27/Downloads/01.데이터/2.Validation/라벨링데이터'

In [3]:
# 데이터 번호와 이름을 출력
print("데이터 번호를 선택하세요:")
for idx, folder_name in enumerate(folder_list, start=1):
    readable_name = folder_name.split('.')[1]  # 폴더명에서 숫자 제외
    print(f"{idx}: {readable_name}")

# 사용자 입력
folder_index = int(input("번호: ")) - 1
if folder_index < 0 or folder_index >= len(folder_list):
    raise ValueError("1~10 사이의 값을 입력해야 합니다!")

print("\n\n유형을 선택하세요 (1: 학습 데이터, 2: 시험 데이터):")
data_type = int(input("번호: "))
if data_type not in [1, 2]:
    raise ValueError("1 또는 2를 입력해야 합니다!")
    
# 데이터 경로 및 폴더 선택
selected_folder = folder_list[folder_index]
base_folder_path = train_folder_path if data_type == 1 else valid_folder_path

데이터 번호를 선택하세요:
1: news_r
2: briefing
3: his_cul
4: paper
5: minute
6: edit
7: public
8: speech
9: literature
10: narration



KeyboardInterrupt



In [16]:
# 데이터를 저장할 빈 리스트
data = []

# JSON 데이터 읽기 및 처리
folder_path = os.path.join(base_folder_path, selected_folder, '2~3sent')
if os.path.exists(folder_path):  # 폴더가 존재하는지 확인
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                json_data = json.load(file)
                passage = json_data.get('Meta(Refine)', {}).get('passage', '')
                summary = json_data.get('Annotation', {}).get('summary1', '')

                data.append({
                    'news': passage,
                    'summary': summary
                })
else:
    raise FileNotFoundError(f"선택한 폴더 {folder_path}가 존재하지 않습니다!")

# 데이터프레임 생성
df = pd.DataFrame(data)

# 현재 시간으로 파일 이름 생성
now = datetime.now().strftime('%H%M')
data_type_str = "train" if data_type == 1 else "valid"
folder_name = selected_folder.split('.')[1]  # "01.news_r"에서 "news_r" 추출
tsv_file_name = f"dataset_{data_type_str}_{folder_name}_{now}.tsv"

# TSV 파일 저장
df.to_csv(tsv_file_name, index=False, encoding='utf-8-sig', sep='\t')

print(f"TSV 파일이 성공적으로 생성되었습니다: {tsv_file_name}")

TSV 파일이 성공적으로 생성되었습니다: dataset_valid_literature_1229.tsv


## 전체 데이터(json)를 tsv로 변환

In [18]:
import os
import json
import pandas as pd
from datetime import datetime
from tqdm import tqdm

# 폴더 리스트 설정
folder_list = [
    "01.news_r",
    "02.briefing",
    "03.his_cul",
    "04.paper",
    "05.minute",
    "06.edit",
    "07.public",
    "08.speech",
    "09.literature",
    "10.narration"
]

# 경로 설정
train_folder_path = 'C:/Users/kmn27/Downloads/01.데이터/1.Training/라벨링데이터'
valid_folder_path = 'C:/Users/kmn27/Downloads/01.데이터/2.Validation/라벨링데이터'

# 데이터를 저장할 빈 리스트
train_data = []
valid_data = []

# JSON 데이터 읽기 및 처리
def process_json_files(base_folder_path, data_list):
    for folder in tqdm(folder_list, desc="Processing folders"):
        folder_path = os.path.join(base_folder_path, folder, '2~3sent')
        if os.path.exists(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith('.json'):
                    file_path = os.path.join(folder_path, filename)
                    with open(file_path, 'r', encoding='utf-8') as file:
                        json_data = json.load(file)
                        passage = json_data.get('Meta(Refine)', {}).get('passage', '')
                        summary = json_data.get('Annotation', {}).get('summary1', '')
                        data_list.append({
                            'news': passage,
                            'summary': summary
                        })
        else:
            print(f"폴더 {folder_path}가 존재하지 않습니다.")

# 학습 및 시험 데이터 처리
print("학습 데이터 처리 중...")
process_json_files(train_folder_path, train_data)

print("\n시험 데이터 처리 중...")
process_json_files(valid_folder_path, valid_data)

# 데이터프레임 생성
train_df = pd.DataFrame(train_data)
valid_df = pd.DataFrame(valid_data)

# 현재 시간으로 파일 이름 생성
now = datetime.now().strftime('%H%M')
train_tsv_file_name = f"dataset_train_all_{now}.tsv"
valid_tsv_file_name = f"dataset_valid_all_{now}.tsv"

# TSV 파일 저장
train_df.to_csv(train_tsv_file_name, index=False, encoding='utf-8-sig', sep='\t')
valid_df.to_csv(valid_tsv_file_name, index=False, encoding='utf-8-sig', sep='\t')

print(f"학습용 TSV 파일이 성공적으로 생성되었습니다: {train_tsv_file_name}")
print(f"시험용 TSV 파일이 성공적으로 생성되었습니다: {valid_tsv_file_name}")


학습 데이터 처리 중...


Processing folders: 100%|██████████| 10/10 [00:06<00:00,  1.64it/s]



시험 데이터 처리 중...


Processing folders: 100%|██████████| 10/10 [00:00<00:00, 12.90it/s]


학습용 TSV 파일이 성공적으로 생성되었습니다: dataset_train_all_1247.tsv
시험용 TSV 파일이 성공적으로 생성되었습니다: dataset_valid_all_1247.tsv


In [6]:
# 데이터 경로
test_path = "C:/Users/USER/OneDrive/onedrive/학교/2024-3학년-2학기/기계학습/final/dataset/test.tsv"

# tsv 파일 읽기
test_data = pd.read_csv(test_path, sep='\t', header=None, names=["news", "summary"])
print("데이터 수:", len(test_data))

print(test_data.sample(1))

데이터 수: 9151
                                                   news  \
3635  김진표 위원] "본 위원은 정부에서 일할 때 우리 군에서 지속적으로 총기 사고나 자...   

                                                summary  
3635  김 위원이 군에서 지속적으로 자살사고나 총기 사고가 일어나는 원인은 좋은 여건에서 ...  
